In [8]:
import datetime as dt
from dateutil.relativedelta import relativedelta
import os
import pandas as pd 
import pandas_datareader as pdr 
import numpy as np 
import quantstats as qs 
import webbrowser as web 
import yfinance as yf

In [36]:
def sma_cross_strategy(ticker, slow=200, fast=50, end=None, period=3):
    if not end:
        end = dt.date.today()
    start = end - relativedelta(years=period)

    data = pd.DataFrame(yf.download(ticker, start=start, end=end)["Close"]) # Get Close price and store in dataframe
    data[f'{fast}-day'] = data.Close.rolling(fast).mean()
    data[f'{slow}-day'] = data.Close.rolling(slow).mean()
    data['returns'] = np.log(data.Close).diff()
    data['strategy'] = np.where(data[f'{fast}-day'] > data[f'{slow}-day'], 1, 0) # Long when fast closes above slow 
    data['strategy'] = np.where(data[f'{fast}-day'] < data[f'{slow}-day'], -1, data['strategy']) # Short when fast closes below slow
    strategy = data['returns'] * data['strategy']

    return strategy


In [41]:
# Example gold investment
gld_cross = sma_cross_strategy("GLD", slow=21, fast=9, period=3)
gld_cross.index = gld_cross.index.tz_localize(None)
gld = qs.utils.download_returns("GLD", period='3y')
gld.index = gld.index.tz_localize(None)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [42]:
qs.extend_pandas()

qs.reports.html(gld_cross, gld, output="gld_cross.html",
                download_filename="gld_cross.html")
web.open_new(f"file:///{os.getcwd()}/gld_cross.html")

d:\Python311\Lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
d:\Python311\Lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
d:\Python311\Lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: The behavior of DataFrame.prod with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


True